# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("Weatherpy/cities.csv")
cities_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,albany,75.0,US,1.582838e+09,47.0,42.60,-73.97,1.67,9.80
1,1,iqaluit,20.0,CA,1.582838e+09,68.0,63.75,-68.51,-27.78,1.00
2,2,mar del plata,0.0,AR,1.582838e+09,30.0,-38.00,-57.56,31.67,7.20
3,3,hofn,100.0,IS,1.582838e+09,78.0,64.25,-15.21,0.01,14.18
4,4,punta arenas,20.0,CL,1.582838e+09,54.0,-53.15,-70.92,14.00,16.50
...,...,...,...,...,...,...,...,...,...,...
572,572,kidal,0.0,ML,1.582838e+09,8.0,18.44,1.41,21.86,3.91
573,573,kralendijk,75.0,BQ,1.582838e+09,62.0,12.15,-68.27,30.00,5.10
574,574,coahuayana,40.0,MX,1.582838e+09,54.0,18.73,-103.68,27.00,2.60
575,575,kloulklubed,90.0,PW,1.582838e+09,88.0,7.04,134.26,26.00,5.10


In [3]:
cities_df = cities_df.drop(columns='Unnamed: 0')
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,albany,75.0,US,1.582838e+09,47.0,42.60,-73.97,1.67,9.80
1,iqaluit,20.0,CA,1.582838e+09,68.0,63.75,-68.51,-27.78,1.00
2,mar del plata,0.0,AR,1.582838e+09,30.0,-38.00,-57.56,31.67,7.20
3,hofn,100.0,IS,1.582838e+09,78.0,64.25,-15.21,0.01,14.18
4,punta arenas,20.0,CL,1.582838e+09,54.0,-53.15,-70.92,14.00,16.50
...,...,...,...,...,...,...,...,...,...
572,kidal,0.0,ML,1.582838e+09,8.0,18.44,1.41,21.86,3.91
573,kralendijk,75.0,BQ,1.582838e+09,62.0,12.15,-68.27,30.00,5.10
574,coahuayana,40.0,MX,1.582838e+09,54.0,18.73,-103.68,27.00,2.60
575,kloulklubed,90.0,PW,1.582838e+09,88.0,7.04,134.26,26.00,5.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

locations = cities_df[["Lat", "Lng"]]
clouds = cities_df['Cloudiness']
humidity = cities_df['Humidity']
max_temp = cities_df['Max Temp']
wind_speed = cities_df['Wind Speed']

In [17]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [57]:
ideal_weather = cities_df.loc[max_temp < 30].loc[max_temp > 10].loc[wind_speed > 8].loc[clouds > 20]
ideal_weather['Lat'] = pd.to_numeric(ideal_weather['Lat'])
ideal_weather['Lng'] = pd.to_numeric(ideal_weather['Lng'])
ideal_weather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
79,port alfred,94.0,ZA,1.582838e+09,95.0,-33.59,26.89,16.67,8.42
96,butaritari,95.0,KI,1.582838e+09,80.0,3.07,172.79,29.10,9.02
154,richards bay,100.0,ZA,1.582838e+09,81.0,-28.78,32.04,25.85,10.64
183,kununurra,90.0,AU,1.582838e+09,94.0,-15.77,128.73,26.00,8.70
237,conceicao da barra,45.0,BR,1.582838e+09,82.0,-18.59,-39.73,27.52,10.00
244,praia da vitoria,40.0,PT,1.582838e+09,93.0,38.73,-27.07,17.00,8.12
418,sulangan,90.0,PH,1.582838e+09,80.0,11.14,123.72,26.61,9.86
520,mogadishu,100.0,SO,1.582838e+09,84.0,2.04,45.34,27.20,9.13


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [62]:
hotel_df = ideal_weather
hotel_df['Hotel Name'] = ""

City           object
Cloudiness    float64
Country        object
Date          float64
Humidity      float64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
Hotel Name     object
dtype: object

In [72]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type":"lodging","key":g_key,"radius":5000}

for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lng = row['Lng']
    params['location'] = lat, lng

    print(f"Retrieving Results for: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # results = response['results']
    print(response)

    # try:
    #     # print(f"Closest hotel is {results[0]['name']}.")     
    #     # hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    # except (KeyError, IndexError):
    #     print("Missing field/result... skipping.")
        
    # print("------------")

Retrieving Results for: port alfred.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Retrieving Results for: butaritari.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Retrieving Results for: richards bay.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Retrieving Results for: kununurra.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Retrieving Results for: conceicao da barra.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Retrieving Results for: praia da vitoria.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Retrieving Results for: sulangan.


In [44]:
hotel_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
79,port alfred,94.0,ZA,1.582838e+09,95.0,-33.59,26.89,16.67,8.42,
96,butaritari,95.0,KI,1.582838e+09,80.0,3.07,172.79,29.10,9.02,
154,richards bay,100.0,ZA,1.582838e+09,81.0,-28.78,32.04,25.85,10.64,
183,kununurra,90.0,AU,1.582838e+09,94.0,-15.77,128.73,26.00,8.70,
237,conceicao da barra,45.0,BR,1.582838e+09,82.0,-18.59,-39.73,27.52,10.00,
244,praia da vitoria,40.0,PT,1.582838e+09,93.0,38.73,-27.07,17.00,8.12,
418,sulangan,90.0,PH,1.582838e+09,80.0,11.14,123.72,26.61,9.86,
520,mogadishu,100.0,SO,1.582838e+09,84.0,2.04,45.34,27.20,9.13,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', scale=2, info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display Map
fig